# Deliverable 2. Create a Customer Travel Destinations Map

----

1. Create a folder called `Vacation_Search` to save all the files related with this deliverable.

2. Download the `Vacation_Search_starter_code.ipynb` Jupyter notebook,save it into your `Vacation_Search folder`, and rename it as `Vacation_Search.ipynb`.

3. In the `Vacation_Search.ipynb` file, ensure that the dependencies and the Geoapify API key is imported correctly.

4. From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`.

5. Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

6. Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`.

7. Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

8. Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

9. Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

10. Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

11. Use a for loop to iterate through the `hotel_df` DataFrame. Retrieve the latitude and longitude of each city and use the Geoapify API to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, set the hotel name as "No hotel found".

12. Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

13. Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

14. Use GeoViews to create a map that displays a point for every city in the `clean_hotel_df` DataFrame. In the point for each city add:

    * The city name
    
    * The country code
    
    * The weather description
    
    * The point's size should be the maximum temperature for the city

15. Take a screenshot of your map and save it to the `Vacation_Search` folder as `WeatherPy_vacation_map.png`.

---

## Ensure that the dependencies and the Geoapify API key is imported correctly

In [4]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import API key
from configeo import geoapify_key

## From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`

In [13]:
# Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database.csv")


# Display sample data
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Mataura,NZ,-46.1927,168.8643,57.47,53,97,4.18,overcast clouds
1,1,Hilo,US,19.7297,-155.0900,75.27,92,53,4.00,broken clouds
2,2,Atar,MR,20.5169,-13.0499,66.65,39,5,9.48,clear sky
3,3,Bathsheba,BB,13.2167,-59.5167,79.41,90,75,19.57,broken clouds
4,4,Hobart,AU,-42.8794,147.3294,68.27,61,20,13.80,few clouds


## Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation

In [14]:
# Prompt the user to enter minimum and maximum temperature criteria.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


## Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`

In [15]:
# Filter the city_data_df DataFrame to find the cities that fit the criteria using the loc method.
preferred_cities_df = city_data_df.loc[
    (city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)
]


# Display sample data
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Hilo,US,19.7297,-155.0900,75.27,92,53,4.00,broken clouds
3,3,Bathsheba,BB,13.2167,-59.5167,79.41,90,75,19.57,broken clouds
6,6,Samarai,PG,-10.6167,150.6667,84.52,71,96,10.29,light rain
8,8,Karratha,AU,-20.7377,116.8463,84.99,56,5,17.47,clear sky
9,9,Arraial Do Cabo,BR,-22.9661,-42.0278,72.82,88,100,6.08,overcast clouds
11,11,Sucila,MX,21.1500,-88.3167,71.62,90,73,3.31,light rain
12,12,Atuona,PF,-9.8000,-139.0333,78.46,79,37,21.25,scattered clouds
14,14,Vanimo,PG,-2.6741,141.3028,82.80,78,100,3.71,overcast clouds
18,18,Verrettes,HT,19.0500,-72.4667,70.74,82,43,2.73,scattered clouds
22,22,Puerto Ayora,EC,-0.7393,-90.3518,70.56,95,93,5.68,overcast clouds


## Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows

In [23]:
# Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
if preferred_cities_df.isnull().values.any() == True:
    preferred_cities_df = preferred_cities_df.dropna()
    clean_travel_cities_df = preferred_cities_df

# Display sample data
clean_travel_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Hilo,US,19.7297,-155.0900,75.27,92,53,4.00,broken clouds
3,3,Bathsheba,BB,13.2167,-59.5167,79.41,90,75,19.57,broken clouds
6,6,Samarai,PG,-10.6167,150.6667,84.52,71,96,10.29,light rain
8,8,Karratha,AU,-20.7377,116.8463,84.99,56,5,17.47,clear sky
9,9,Arraial Do Cabo,BR,-22.9661,-42.0278,72.82,88,100,6.08,overcast clouds
...,...,...,...,...,...,...,...,...,...,...
692,692,Cabo Rojo,PR,18.0866,-67.1457,75.96,65,0,5.28,clear sky
694,694,Sao Jose Da Coroa Grande,BR,-8.8978,-35.1478,76.86,77,83,7.16,broken clouds
697,697,Taua,BR,-5.9667,-40.3000,75.88,58,78,6.76,broken clouds
701,701,Port Macquarie,AU,-31.4333,152.9167,70.90,58,55,17.11,broken clouds


## Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng"

In [31]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_travel_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df


,City,Country,Max Temp,Lat,Lng
1,Hilo,US,75.27,19.7297,-155.0900
3,Bathsheba,BB,79.41,13.2167,-59.5167
6,Samarai,PG,84.52,-10.6167,150.6667
8,Karratha,AU,84.99,-20.7377,116.8463
9,Arraial Do Cabo,BR,72.82,-22.9661,-42.0278
...,...,...,...,...,...
692,Cabo Rojo,PR,75.96,18.0866,-67.1457
694,Sao Jose Da Coroa Grande,BR,76.86,-8.8978,-35.1478
697,Taua,BR,75.88,-5.9667,-40.3000
701,Port Macquarie,AU,70.90,-31.4333,152.9167


## Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame

In [32]:
# Create a new column "Hotel Name".
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Hilo,US,75.27,19.7297,-155.0900,
3,Bathsheba,BB,79.41,13.2167,-59.5167,
6,Samarai,PG,84.52,-10.6167,150.6667,
8,Karratha,AU,84.99,-20.7377,116.8463,
9,Arraial Do Cabo,BR,72.82,-22.9661,-42.0278,
...,...,...,...,...,...,...
692,Cabo Rojo,PR,75.96,18.0866,-67.1457,
694,Sao Jose Da Coroa Grande,BR,76.86,-8.8978,-35.1478,
697,Taua,BR,75.88,-5.9667,-40.3000,
701,Port Macquarie,AU,70.90,-31.4333,152.9167,


## Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city

In [33]:
# Set parameters to search for a hotel
radius = 5000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

## Use a for loop to iterate through the `hotel_df` DataFrame. Retrieve the latitude and longitude of each city and use the Geoapify API to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, set the hotel name as "No hotel found"

In [34]:
# Iterate through the hotel DataFrame

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from DataFrame.
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set up the base URL for the Geoapify Places API.
    base_url = "https://api.geoapify.com/v2/places"

    # Make request and retrieve the JSON data by using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found set the hotel name as "No hotel found".
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
Hilo - nearest hotel: Dolphin Bay Hotel
Bathsheba - nearest hotel: Atlantis Hotel
Samarai - nearest hotel: No hotel found
Karratha - nearest hotel: Karratha International Hotel
Arraial Do Cabo - nearest hotel: No hotel found
Sucila - nearest hotel: No hotel found
Atuona - nearest hotel: Pearl Resort
Vanimo - nearest hotel: CBC Guesthouse
Verrettes - nearest hotel: Kay Haiti
Puerto Ayora - nearest hotel: Hostal La Mirada De Solitario George
Codrington - nearest hotel: No hotel found
San Policarpo - nearest hotel: No hotel found
Abu Dhabi - nearest hotel: Marriott Downtown Abu-Dhabi
Rikitea - nearest hotel: Chez Bianca & Benoit
Kaeo - nearest hotel: No hotel found
Saint-Philippe - nearest hotel: Le Baril
Minab - nearest hotel: هتل صدف
Yulara - nearest hotel: Outback Pioneer Hotel & Lodge
Busselton - nearest hotel: No hotel found
Tonj - nearest hotel: No hotel found
Avarua - nearest hotel: Paradise Inn
Vaini - nearest hotel: Keleti International Resort
Port Alfred - 

Taganak - nearest hotel: No hotel found
Chigorodo - nearest hotel: Gran Hotel Principal
Vila Do Maio - nearest hotel: Residencial Jardins De Maio
Sao Filipe - nearest hotel: Bela Vista
Wangaratta - nearest hotel: Hotel Pinsent
Bontang - nearest hotel: Hotel Sederhana
Mareeba - nearest hotel: The Gateway Hotel
Bandarbeyla - nearest hotel: No hotel found
Saint-Joseph - nearest hotel: No hotel found
Grand-Santi - nearest hotel: No hotel found
Les Trois-Bassins - nearest hotel: No hotel found
Phangnga - nearest hotel: Hotel
Vao - nearest hotel: Nataiwatch
Daru - nearest hotel: Kuki
Adiake - nearest hotel: No hotel found
Asifabad - nearest hotel: bhagyalaxmi hotel
San Andres - nearest hotel: Hotel Decameron Isleño
Bongandanga - nearest hotel: No hotel found
Balabac - nearest hotel: No hotel found
Santa Cruz - nearest hotel: Aviador Hotel Boutique
Tautira - nearest hotel: No hotel found
Oriximina - nearest hotel: No hotel found
Hobyo - nearest hotel: No hotel found
Ambalavao - nearest hotel:

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Hilo,US,75.27,19.7297,-155.0900,Dolphin Bay Hotel
3,Bathsheba,BB,79.41,13.2167,-59.5167,Atlantis Hotel
6,Samarai,PG,84.52,-10.6167,150.6667,No hotel found
8,Karratha,AU,84.99,-20.7377,116.8463,Karratha International Hotel
9,Arraial Do Cabo,BR,72.82,-22.9661,-42.0278,No hotel found
...,...,...,...,...,...,...
692,Cabo Rojo,PR,75.96,18.0866,-67.1457,No hotel found
694,Sao Jose Da Coroa Grande,BR,76.86,-8.8978,-35.1478,Hotel do Sol
697,Taua,BR,75.88,-5.9667,-40.3000,No hotel found
701,Port Macquarie,AU,70.90,-31.4333,152.9167,The Observatory


## Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`

In [36]:
# Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

# Display sample data
clean_hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Hilo,US,75.27,19.7297,-155.0900,Dolphin Bay Hotel
3,Bathsheba,BB,79.41,13.2167,-59.5167,Atlantis Hotel
6,Samarai,PG,84.52,-10.6167,150.6667,No hotel found
8,Karratha,AU,84.99,-20.7377,116.8463,Karratha International Hotel
9,Arraial Do Cabo,BR,72.82,-22.9661,-42.0278,No hotel found
...,...,...,...,...,...,...
692,Cabo Rojo,PR,75.96,18.0866,-67.1457,No hotel found
694,Sao Jose Da Coroa Grande,BR,76.86,-8.8978,-35.1478,Hotel do Sol
697,Taua,BR,75.88,-5.9667,-40.3000,No hotel found
701,Port Macquarie,AU,70.90,-31.4333,152.9167,The Observatory


## Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the `Vacation_Search`` folder

In [37]:
# Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_Id")

## Use GeoViews to create a map that displays a point for every city in the `clean_hotel_df` DataFrame.

In the point for each city add:

* The city name
* The country code
* The weather description
* The point's size should be the maximum temperature for the city

In [40]:
# Configure the map plot
# The city name

# The country code

# The weather description

# The point's size should be the maximum temperature for the city

hotel_map = clean_hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Max Temp",
    scale = 1,
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Hotel Name,Country)